In [ ]:
import ee
import os
import pandas as pd

# ==========================================
# PART 1: SETUP & EARTH ENGINE INITIALIZATION
# ==========================================
# ee.Authenticate()
ee.Initialize()

# CONFIGURATION
START_DATE = '2022-05-01'
END_DATE   = '2024-11-01'
LOCAL_DATA_DIR = 'data'

In [ ]:
# ==========================================
# PART 2: DATA EXTRACTION (EARTH ENGINE)
# ==========================================

full_area = ee.FeatureCollection('projects/ee-zuruyuyu/assets/airesanteNK')

names_to_keep = [
    "nk Masisi Aire de Santé", "nk Kitsombiro Aire de Santé",
    "nk Kirumba2 Aire de Santé", "nk CCLK Aire de Santé", "nk Nyasi Aire de Santé",
    "nk Vusenzera Aire de Santé", "nk Sake Aire de Santé", "nk Masuli Aire de Santé",
    "nk Murara Aire de Santé", "nk Kiziba Aire de Santé", "nk Lubero Cité Aire de Santé",
    "nk Vungi Aire de Santé", "nk Virunga Aire de Santé", "nk Umoja Aire de Santé",
    "nk Kyondo Aire de Santé", "nk Kirotshe Aire de Santé", "nk Kipese Aire de Santé",
    "nk Malepe Aire de Santé", "nk Heal Africa Aire de Santé", "nk Kyangike Aire de Santé",
    "nk Kanzulinzuli Aire de Santé", "nk Rutshuru Aire de Santé"
]
selected_area = full_area.filter(ee.Filter.inList('name', names_to_keep))

def to_celsius(image):
    return (image
            .select('skin_temperature')
            .subtract(273.15)
            .rename('Td')
            .copyProperties(image, image.propertyNames()))

dataset = (ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")
           .filterDate(START_DATE, END_DATE)
           .map(to_celsius))

def export_to_drive(fc, description, filename):
    def reduce_over_features(image):
        date_str = image.date().format('YYYY-MM-dd')
        stats = image.select('Td').reduceRegions(
            collection=fc,
            reducer=ee.Reducer.mean(),
            scale=1000
        )
        return stats.map(lambda f: f.set('date', date_str))

    flat_fc = dataset.map(reduce_over_features).flatten()

    task = ee.batch.Export.table.toDrive(
        collection=flat_fc,
        description=description,
        fileNamePrefix=filename,
        fileFormat='CSV',
        selectors=['name', 'date', 'mean']
    )
    task.start()
    print(f"Task started: {description}")

export_to_drive(full_area, 'Td_results_full_raw', 'Td_results_full_raw')
export_to_drive(selected_area, 'Td_results_selected_raw', 'Td_results_selected_raw')

In [ ]:
# ==========================================
# PART 3: DATA ANALYSIS (PANDAS)
# ==========================================
# Only run this part after downloading the CSVs to your local 'data' folder

def process_heatwaves_td():
    full_path = os.path.join(LOCAL_DATA_DIR, 'Td_results_full_raw.csv')
    sel_path = os.path.join(LOCAL_DATA_DIR, 'Td_results_selected_raw.csv')

    if not os.path.exists(full_path) or not os.path.exists(sel_path):
        print("\n[INFO] CSV files not found locally. Skipping analysis step.")
        print("Please download the exported files from Google Drive to the 'data' folder.")
        return

    print("\nProcessing CSVs...")
    df_full = pd.read_csv(full_path)
    df_sel = pd.read_csv(sel_path)

    for df in [df_full, df_sel]:
        df['date'] = pd.to_datetime(df['date'])
        df.sort_values(['name', 'date'], inplace=True)

    def mark_heatwave(group):
        group = group.reset_index(drop=True)
        flags = ['no'] * len(group)
        run_start = None

        for i, val in enumerate(group['mean']):
            if pd.notna(val) and val > 30:
                if run_start is None:
                    run_start = i
            else:
                if run_start is not None:
                    if i - run_start >= 3:
                        flags[run_start:i] = ['yes'] * (i - run_start)
                    run_start = None

        if run_start is not None and len(group) - run_start >= 3:
            flags[run_start:] = ['yes'] * (len(group) - run_start)

        group['heatwave'] = flags
        return group

    df_full = df_full.groupby('name', group_keys=False).apply(mark_heatwave)
    df_sel = df_sel.groupby('name', group_keys=False).apply(mark_heatwave)

    df_full.to_csv(os.path.join(LOCAL_DATA_DIR, 'Td_results_full_heatwave.csv'), index=False)
    df_sel.to_csv(os.path.join(LOCAL_DATA_DIR, 'Td_results_selected_heatwave.csv'), index=False)
    print("Analysis complete. Files saved to 'data' folder.")

if __name__ == "__main__":
    process_heatwaves_td()